In [1]:
import pandas as pd
import sqlalchemy
import pandas_ta
from sklearn.preprocessing import normalize, StandardScaler
from sklearn.metrics import confusion_matrix
import xgboost

import pandas as pd
import sqlalchemy
import pandas_ta
from sklearn.preprocessing import normalize, StandardScaler
from sklearn.metrics import confusion_matrix

def ticker_preprocess(result_set) -> pd.DataFrame:
    # print('current ticker group {}'.format(resultSet))
    norm_columns = ['Adj Close','Volume', 'MACD_12_26_9' ,'MACDh_12_26_9' ,'MACDs_12_26_9' ,'RSI_14' ,"BBL_5_2.0" ,"BBM_5_2.0" ,"BBU_5_2.0" ,"BBB_5_2.0" ,"BBP_5_2.0" ,'OBV' ,'SMA_20' ,'EMA_50' ,'STOCHk_14_3_3' ,'STOCHd_14_3_3' ,'ADX_14' ,'DMP_14' ,'DMN_14' ,'AD','delta_p5', 'delta_p20', 'delta_p90', 'delta_v5', 'delta_v20', 'delta_v90', 'PV_DELTA_R5','PV_DELTA_R20','PV_DELTA_R90',
                    'PV_DELTA_R20_PRIOR1', 'PV_DELTA_R20_PRIOR2','PV_DELTA_R20_PRIOR3', 'PV_DELTA_R20_PRIOR4', 'PV_DELTA_R20_PRIOR5', 'MFI_14']
    result_set.ta.vwma(close='Adj Close', append=True, length=5)
    result_set.ta.vwma(close='Adj Close', append=True, length=20)
    result_set.ta.vwma(close='Adj Close', append=True, length=90)
    result_set.ta.mfi(close='Adj Close', append=True)
    result_set['Volume_5'] = result_set['Volume'].rolling(5).mean()
    result_set['Volume_20'] = result_set['Volume'].rolling(20).mean()
    result_set['Volume_90'] = result_set['Volume'].rolling(90).mean()

    result_set['delta_p5'] = result_set['Adj Close'] - result_set['VWMA_5']
    result_set['delta_p20'] = result_set['Adj Close'] - result_set['VWMA_20']
    result_set['delta_p90'] = result_set['Adj Close'] - result_set['VWMA_90']

    result_set['delta_v5'] = result_set['Volume'] - result_set['Volume_5']
    result_set['delta_v20'] = result_set['Volume'] - result_set['Volume_20']
    result_set['delta_v90'] = result_set['Volume'] - result_set['Volume_90']
    
    result_set['PV_DELTA_R5'] = result_set['delta_p5'] /result_set['delta_v5'] 
    result_set['PV_DELTA_R20'] = result_set['delta_p20'] /result_set['delta_v20'] 
    result_set['PV_DELTA_R90'] = result_set['delta_p90'] /result_set['delta_v90'] 

    result_set['PV_DELTA_R20_PRIOR1'] = result_set['PV_DELTA_R20'].shift(1)
    result_set['PV_DELTA_R20_PRIOR2'] = result_set['PV_DELTA_R20'].shift(2)
    result_set['PV_DELTA_R20_PRIOR3'] = result_set['PV_DELTA_R20'].shift(3)
    result_set['PV_DELTA_R20_PRIOR4'] = result_set['PV_DELTA_R20'].shift(4)
    result_set['PV_DELTA_R20_PRIOR5'] = result_set['PV_DELTA_R20'].shift(5)
    
    result_set['PRESSURE_FORECAST'] = result_set['PRESSURE_AVG'].rolling(5).mean()
    result_set['TEMPERATURE_FORECAST'] = result_set['TEMPERATURE_AVG'].rolling(5).mean()
    
    result_set['target'] = (result_set['Y_CLOSE5_MAX'] - result_set['Adj Close'] ) / result_set['Adj Close'] * 100 > 5
            
            # ( result_set['Y_CLOSE5_MAX'] - result_set['Adj Close'] ) / result_set['Adj Close'] * 100)
    
    sd_scaler = StandardScaler()
    result_set[norm_columns] = sd_scaler.fit_transform(result_set[norm_columns])
    return result_set

# please use the dataset bank-additional.zip and extract it
query = '''
select DISTINCT S.date,Open ,High ,Low ,Close ,"Adj Close" ,Volume ,MACD_12_26_9 ,MACDh_12_26_9 ,MACDs_12_26_9 ,RSI_14 ,"BBL_5_2.0" ,
"BBM_5_2.0" ,"BBU_5_2.0" ,"BBB_5_2.0" ,"BBP_5_2.0" ,OBV ,SMA_20 ,EMA_50 ,STOCHk_14_3_3 ,STOCHd_14_3_3 ,ADX_14 ,DMP_14 ,DMN_14 ,AD ,
STDEV_30  ,Y_CLOSE5_MAX,ticker,BULLISH,BEARISH, TEMPERATURE_AVG,PRESSURE_AVG
 from STOCK_DAILY S
 INNER JOIN AAII_SENTIMENT A ON (date(A.DATE) BETWEEN date(S.DATE, '-7 days') AND date(S.DATE)) 
 INNER JOIN NY_WEATHER NY WHERE NY.DATE = S.DATE AND S.DATE > '2023-06-01'
'''
sqlengine = sqlalchemy.create_engine('sqlite:///D:\\sqlite\\data\\finance.db')
result_set = pd.read_sql_query(query, sqlengine.connect())
grouped = result_set.groupby(['ticker'])
finalResult = pd.DataFrame()

for name, group in grouped:
    # group.reset_index()
    # group.set_index('Date',inplace=True)
    group = ticker_preprocess(group)
    finalResult = pd.concat([group,finalResult])
    
print(finalResult.head())

cols_to_drop = ['Date','Open','High','Low','Close','ticker','VWMA_5','VWMA_20','VWMA_90','Y_CLOSE5_MAX','target']
X = finalResult.drop(columns=cols_to_drop).dropna()
# X.head()


m1 = xgboost.XGBClassifier() 
m1.load_model("./tamed/xgb_m1.json") 

m1.predict(X)


                            Date        Open        High         Low  \
1530  2023-06-01 00:00:00.000000  175.869995  176.449997  172.899994   
1531  2023-06-01 00:00:00.000000  175.869995  176.449997  172.899994   
1532  2023-06-02 00:00:00.000000  176.500000  176.630005  174.100006   
1533  2023-06-05 00:00:00.000000  175.190002  175.190002  169.770004   
1534  2023-06-06 00:00:00.000000  169.399994  171.539993  168.899994   

           Close  Adj Close    Volume  MACD_12_26_9  MACDh_12_26_9  \
1530  175.809998   0.984251 -0.601789      0.788701       1.890856   
1531  175.809998   0.984251 -0.601789      0.788701       1.890856   
1532  175.229996   0.929485 -0.219008      0.880505       1.714484   
1533  170.259995   0.460205  0.118139      0.773180       1.071555   
1534  170.360001   0.469648 -0.566368      0.687039       0.610598   

      MACDs_12_26_9  ...  PV_DELTA_R20  PV_DELTA_R90  PV_DELTA_R20_PRIOR1  \
1530       0.212482  ...           NaN           NaN                 

ValueError: DataFrame.dtypes for data must be int, float, bool or category. When categorical type is supplied, The experimental DMatrix parameter`enable_categorical` must be set to `True`.  Invalid columns:BULLISH: object, BEARISH: object, TEMPERATURE_AVG: object, PRESSURE_AVG: object